# Use google spreadsheet table to collect data

In previous version of the script we combined the manual annotation with the measurements. Now we analyze just the overal google spreadsheet

In [1]:
%load_ext autoreload
%autoreload
!pip install pandas==1.5.3 openpyxl


import numpy as np
import seaborn as sns
import scipy
import skimage

sns.set_style("ticks")
from pathlib import Path

import pandas as pd
import pigleg_evaluation_tools as pet



import numpy as np
import seaborn as sns
import scipy
import skimage

sns.set_style("ticks")
from pathlib import Path

# Get the data
# in this file is the url to the spreadsheet with the data
# PIGLEG_SPREADSHEET_URL=https://docs.google.com/spreadsheets/d/1G55kXxcJ0...piLaK0snddRU-jzU/edit?usp=sharing
env_paths = list(Path(".").glob("../**/.env.dev"))
print(env_paths)
# find line with PIGLEG_SPREADSHEET_URL and give me the value after the "="
PIGLEG_SPREADSHEET_URL = \
[line.split("=", 1)[1].strip() for line in open(env_paths[0], "r") if line.startswith("PIGLEG_SPREADSHEET_URL")][0]
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("\"", "")
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("edit?usp=sharing", "export?format=csv")
# PIGLEG_SPREADSHEET_URL
import pandas as pd

url = PIGLEG_SPREADSHEET_URL
df = pd.read_csv(url)

# Get the data
# fn = sorted(list(Path(".").glob("*list_of_all_stitches*.xlsx")))[-1]
# fn = sorted(list(Path(".").glob("*all_stitches_with_human_annotations*.xlsx")))[-1]  # this works with ADD_ADVANCED_STUDENTS_TO_EXPERTS
# # fn = sorted(list(Path(".").glob("*all_stitches_with_human_annotations.CBA*.xlsx")))[-1]
# print(fn)
# dfall = pd.read_excel(fn)

#remove unnamed columns
dfall = dfall.loc[:, ~dfall.columns.str.contains('^Unnamed')]
ADD_ADVANCED_STUDENTS_TO_EXPERT = True

dfall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 50.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pandas (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [3494 lines of output]
      <string>:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
      /tmp/pip-build-env-2dvvh4bk/overlay/lib/python3.14/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license e

ModuleNotFoundError: No module named 'seaborn'

In [119]:
sorted(list(Path(".").glob("*all_stitches_with_human_annotations*.xlsx")))

[PosixPath('all_stitches_with_human_annotations.2024-09-18_18-58-20.xlsx'),
 PosixPath('all_stitches_with_human_annotations.2024-09-18_18-59-12.xlsx'),
 PosixPath('all_stitches_with_human_annotations.CBA_New_OSATS_rating.2025-01-13_21-16-32.xlsx'),
 PosixPath('all_stitches_with_human_annotations.CBA_New_OSATS_rating.2025-02-13_09-56-24.xlsx'),
 PosixPath('all_stitches_with_human_annotations.CBA_New_OSATS_rating.2025-02-14_21-57-56.xlsx'),
 PosixPath('all_stitches_with_human_annotations.annotators_vs_algorithm_data_ta.2025-01-13_20-11-13.xlsx'),
 PosixPath('all_stitches_with_human_annotations_and_predictions.2024-09-19 00:36:04.xlsx'),
 PosixPath('all_stitches_with_human_annotations_and_predictions.xlsx')]

In [120]:
if ADD_ADVANCED_STUDENTS_TO_EXPERT:
    annotators = ["Carina", "Ana", "Mira"]

    dfall["mean_movement_annotation"] = dfall[annotators].mean(axis=1)
    dfall.mean_movement_annotation.describe()

In [121]:
# pick numeric columns
numeric_columns = dfall.select_dtypes(include=[np.number]).columns


# Remove NaN values

In [122]:
# kick rows with a lot missing values
missing_values_rows = dfall.isnull().sum(axis=1)
missing_values_rows.sort_values(ascending=False)
kicked_lines = dfall[missing_values_rows > 100]
dfall = dfall.drop(kicked_lines.index)
kicked_lines

,filename,annotation_annotation_annotation,annotation_annotation_time_and_movements,report_url,Stitch duration [s],Stitch start at [s],Needle holder stitch area presence [%],Needle holder stitch median area presence [%],Needle holder stitch length [m],Needle holder stitch visibility [s],...,done_by_expert,file_stitch,Carina,Ana,Mira,mean_movement_annotation,is_train,LinearRegression mean_movement_annotation,MLPRegressor mean_movement_annotation,SVR mean_movement_annotation


In [123]:
# from numeric columns remove columns with lot of missing values
missing_values_columns = dfall[numeric_columns].isnull().sum()
missing_values_columns.sort_values(ascending=False)
kicked_columns = missing_values_columns[missing_values_columns > 20]
dfall = dfall.drop(kicked_columns.index, axis=1)
dfall.shape

(87, 121)

In [124]:

if ADD_ADVANCED_STUDENTS_TO_EXPERT:
    relevant_column = "mean_movement_annotation"
    annotation_columns = annotators + ["mean_movement_annotation", "movement evaluation stitch", "done_by_expert", "annotation_annotation_time_and_movements"]
    id_numeric_columns = ["stitch_id"]
    sample_id_cols = ["filename", "stitch_id"]
    predicted_columns = [relevant_column]  # list(corr.columns[corr.columns.str.contains("is_pretest_num")])
    predicted_columns_extended = [relevant_column]  + annotators

    numeric_columns = dfall.select_dtypes(include=[np.number]).columns
    measured_columns = numeric_columns.drop(annotation_columns + id_numeric_columns)
    dfall.shape

In [125]:

    # dfall["done_by_expert"] = dfall["filename"].str.contains("Ekn").astype(float)
    # dfall["done_by_expert"] = dfall["done_by_expert"].astype(bool)

# First we used only videos uploaded by Ana. They contain Ekn
# Later we learned, that videos starting with Ekn_CSW are not experts videos
# We completely forgot to include videos by Carina. They contain EKN_{number}_CBA

# dfall["done_by_expert"] = dfall["filename"].str.contains(r"Ekn", regex=True).astype(float)
dfall["done_by_expert"] = dfall["filename"].str.contains(r"Ekn_Ana", regex=True).astype(bool) | dfall["filename"].str.contains(r"EKN_\d+_CBA", regex=True)
# dfall["done_by_expert"] = dfall["filename"].str.contains(r"EKN_\d+_CBA", regex=True)

# done_by_expert = dfall["filename"].str.contains("Ekn_Ana")
dfall["done_by_expert"].value_counts()


False    64
True     23
Name: done_by_expert, dtype: int64

In [126]:
# measured_columns

In [127]:
# measured_columns.append("mean_movement_annotation")

In [128]:
# correlation of all columns with the mean_movement_annotation
# correlations

In [129]:
dfst = dfall

if ADD_ADVANCED_STUDENTS_TO_EXPERT:
    corr = dfall[list(measured_columns) + [relevant_column]].corr()["mean_movement_annotation"].abs().sort_values(ascending=False).head(21)
    # corr = dfst.corr(numeric_only=True)[relevant_column].abs().sort_values(ascending=False).head(10)
    corr = corr[corr.index != relevant_column]
    cols_X = list(corr.index)
    corr


# Separate good and bad students


In [130]:
# find the threshold beteween experts and students based on KDE model of each group descriped by done_by_expert
import plotly.express as px

if ADD_ADVANCED_STUDENTS_TO_EXPERT:
    med = dfst["mean_movement_annotation"].median()
    # find the threshold beteween experts and students based on KDE model of each group descriped by done_by_expert
    kde_expert = scipy.stats.gaussian_kde(dfst[dfst.done_by_expert==1]["mean_movement_annotation"].dropna())
    kde_student = scipy.stats.gaussian_kde(dfst[dfst.done_by_expert==0]["mean_movement_annotation"].dropna())



    # find the intersection of the two KDEs
    from scipy.optimize import fsolve
    def findIntersection(fun1, fun2, x0):
        return fsolve(lambda x : fun1(x) - fun2(x), x0)
    intersection = findIntersection(kde_expert, kde_student, med)
    expert_threshold = intersection[0]
    expert_threshold

In [131]:
# with the Ekn_CSW
# expert_threshold = 3.79
# just Ekn_Ana
expert_threshold = 3.9476

In [132]:
if ADD_ADVANCED_STUDENTS_TO_EXPERT:
    movement_evaluation_col = "mean_movement_annotation"


    fig = px.histogram(dfst, x=movement_evaluation_col, title="mean_movement_annotation",
                       color="done_by_expert", opacity=0.5, nbins=100)
    # draw vline
    fig.add_vline(x=expert_threshold, line_width=3, line_dash="dash", line_color="green", annotation_text=f"intersection={expert_threshold:.2f}", annotation_position="top right")

    fig.show()


def add_group_column(dfst, movement_evaluation_col, expert_threshold):
    dfst["Group"] = "expert"
    if ADD_ADVANCED_STUDENTS_TO_EXPERT:
        dfst["Group"][dfst[movement_evaluation_col] < expert_threshold] = "student"
    else:
        dfst["Group"][dfst["done_by_expert"]==False] = "student"


    return dfst

dfst = add_group_column(dfst, movement_evaluation_col, expert_threshold)



# Make suggestion for students

In [133]:

def find_threshold(dfst, col_name):
    med = dfst[col_name].median()
    # find the threshold beteween experts and students based on KDE model of each group descriped by done_by_expert
    kde_expert = scipy.stats.gaussian_kde(dfst[dfst["Group"]=='expert'][col_name].dropna())
    kde_student = scipy.stats.gaussian_kde(dfst[dfst["Group"]=='student'][col_name].dropna())
    # find the intersection of the two KDEs
    intersection = findIntersection(kde_expert, kde_student, med)
    return intersection[0]
 



In [134]:
import plotly.express as px

# col_name = "Needle holder stitch visibility [%]"
# col_name = "Needle holder stitch length [m]"
# col_name = "Stitch duration [s]"
# col_name = "Needle holder stitch median area presence [%]"
# col_name = "Needle holder stitch velocity above threshold"
# col_name = "Needle holder stitch area presence [%]"
# 
# col_name = "Forceps stitch area presence [%]"
# col_name = "Knot duration [s]"

col_names = [
    "Stitch duration [s]" ,
    "Needle holder stitch area presence [%]" ,
    "Needle holder stitch median area presence [%]",
    "Needle holder stitch length [m]", 
    "Needle holder stitch visibility [%]",
    "Needle holder stitch velocity above threshold" ,
    "Forceps stitch area presence [%]" ,
    "Forceps stitch median area presence [%]" ,
    "Knot duration [s]",
    "Needle holder to forceps stitch below threshold [s]"
]
# col_name = "mean_movement_annotation"

In [135]:
import plotly.figure_factory as ff
def get_distplot(dfst, col_name, my_value, annotation_text="You"):
    fig = ff.create_distplot([dfst[dfst["Group"]=="student"][col_name].dropna(), dfst[dfst["Group"]=='expert'][col_name].dropna()],
                             group_labels=['student', 'expert'],
                             show_hist=True,
                             show_rug=True,
                             curve_type='kde',
                             # histnorm='probability density'
                             histnorm='probability'
                             )
    # description of x-axis
    fig.update_xaxes(title_text=col_name)
    # legend
    fig.update_layout(legend_title_text='Group')
    # comparison_value
    fig.add_vline(x=my_value, line_width=3, line_dash="dash", line_color="green", annotation_text=annotation_text, annotation_position="top right")
    # arrow to my_value with text
    # fig.add_annotation(x=my_value, y=0.5, text="mean", showarrow=True, arrowhead=1)
    # fig.show()
    return fig


In [136]:
col_names

['Stitch duration [s]',
 'Needle holder stitch area presence [%]',
 'Needle holder stitch median area presence [%]',
 'Needle holder stitch length [m]',
 'Needle holder stitch visibility [%]',
 'Needle holder stitch velocity above threshold',
 'Forceps stitch area presence [%]',
 'Forceps stitch median area presence [%]',
 'Knot duration [s]',
 'Needle holder to forceps stitch below threshold [s]']

In [137]:

for col_name in col_names:

    # my_value = dfst[col_name].mean()
    my_value = find_threshold(dfst, col_name)


    fig = get_distplot(dfst, col_name, my_value, f"{my_value:.2f}")
    fig.show()


In [26]:
# plot cumulative of kde

def show_cumulative(col_name, my_value):
    kde_all = scipy.stats.gaussian_kde(dfst[col_name].dropna())
    x = np.linspace(dfst[col_name].min(), dfst[col_name].max(), 1000)
    y = kde_all(x)
    y_cum = 100* np.cumsum(y) / np.sum(y)
    # give me percentile of my_value
    percentile = np.interp(my_value, x, y_cum)

    fig = px.line(x=x, y=y_cum, title=f"Cumulative distribution of {col_name}", labels={"x": col_name, "y": "cumulative distribution [%]"})
    fig.add_vline(x=my_value, line_width=3, line_dash="dash", line_color="green", annotation_text=f"my_value={int(percentile)} [%]", annotation_position="top right")

    fig.show()

show_cumulative("mean_movement_annotation", my_value=3.6)

for col_name in col_names:
    # my_value = find_threshold(dfst, col_name)
    mean_value = dfst[col_name].mean()
    show_cumulative(col_name, my_value=mean_value)



In [79]:
# drop zeros and NaNs in column col_name
dfst_tmp = dfst.dropna(subset=[col_name])
dfst_tmp = dfst_tmp[dfst_tmp[col_name] != 0]



In [80]:
dfst

,filename,annotation_annotation_annotation,annotation_annotation_time_and_movements,report_url,Stitch duration [s],Stitch start at [s],Needle holder stitch area presence [%],Needle holder stitch median area presence [%],Needle holder stitch length [m],Needle holder stitch visibility [s],...,done_by_expert,file_stitch,Carina,Ana,Mira,mean_movement_annotation,is_train,LinearRegression mean_movement_annotation,MLPRegressor mean_movement_annotation,SVR mean_movement_annotation
0,Ekn_Ana_3.mp4,00:00:01 stitch_start\n00:00:24 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,53.0,1.0,44.822485,89.053254,2.146999,45.066667,...,1,Ekn_Ana_3.mp4_0,4.2,4.333333,5.0,4.511111,0.0,4.069516,4.008933,4.105752
1,Ekn_Ana_2.mp4,00:00:02 stitch_start\n00:00:21 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,50.0,2.0,54.172989,94.992413,1.920482,43.933333,...,1,Ekn_Ana_2.mp4_0,4.6,4.666667,5.0,4.755556,1.0,4.145910,4.019103,4.224466
2,Ekn_Ana_4.mp4,00:00:01 stitch_start\n00:00:21 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,51.0,1.0,61.417323,94.015748,1.920755,42.333333,...,1,Ekn_Ana_4.mp4_0,3.2,3.666667,4.5,3.788889,1.0,4.080613,3.974225,4.131889
3,Ekn_Ana_1.mp4,00:00:01 stitch_start\n00:00:22 knot_end\n00:0...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,48.0,1.0,44.511278,68.270677,3.370340,44.333333,...,1,Ekn_Ana_1.mp4_0,4.4,4.333333,5.0,4.577778,1.0,4.220112,4.156992,4.254250
4,Ekn_Ana_5.mp4,00:00:00 stitch_start\n00:00:17 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,43.0,0.0,36.909091,84.000000,2.854883,36.666667,...,1,Ekn_Ana_5.mp4_0,3.2,3.666667,5.0,3.955556,0.0,4.231069,4.146621,4.305917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Bildschirmaufnahme_2022-10-11_um_11.20.24.mov,00:00:17 stitch_start\n00:01:41 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,121.0,552.0,82.840237,92.504931,3.348492,101.400000,...,0,Bildschirmaufnahme_2022-10-11_um_11.20.24.mov_3,2.6,3.000000,4.0,3.200000,1.0,2.955111,3.033193,2.965313
83,Schiesser_David_post_test.mp4,00:00:04 stitch_start\n00:00:47 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,58.0,195.0,78.386167,93.227666,2.580827,46.266667,...,0,Schiesser_David_post_test.mp4_3,3.8,4.250000,3.0,3.683333,NaN,3.928319,3.947800,4.028566
84,Einzelknopfnaht_Schwein_Kerstin_Preuß.mp4,00:00:06 stitch_start\n00:00:32 knot_start \n0...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,50.0,240.0,83.281250,93.125000,2.302644,42.666667,...,0,Einzelknopfnaht_Schwein_Kerstin_Preuß.mp4_4,3.8,3.500000,4.0,3.766667,NaN,4.120399,3.966669,4.090838
85,Stresstest_Einzelknopfnaht.mp4,Das Ergebnis finde ich sehr schön. Die Wundrän...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,54.0,348.0,89.949749,99.874372,1.301011,53.066667,...,0,Stresstest_Einzelknopfnaht.mp4_4,3.6,3.750000,5.0,4.116667,NaN,4.197564,4.060015,4.140068


In [109]:
# plot KDE distribution of column

fig = px.histogram(dfst, x=col_name, title=col_name, 
                   # color="done_by_expert",
                   color='level',
                   opacity=0.5, 
                   # cumulative=True,
                   histnorm='probability density')
# show legend
fig.update_traces(opacity=0.75)
# add density line
fig.update_traces(histnorm='probability density', nbinsx=100)

# add vertical line for mean
fig.add_vline(x=dfst[col_name].mean(), line_width=3, line_dash="dash", line_color="green", )

fig.show()

AttributeError: module 'plotly.express' has no attribute 'distplot'

In [85]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

# Example DataFrame
df = pd.DataFrame({'your_column': np.random.randn(1000)})

# Extract the column for which we want to calculate the KDE
data = df['your_column']
data = dfst[col_name].dropna()

# Perform the KDE
kde = gaussian_kde(data)
# 90% percentile of data
# percentile_95 = np.percentile(data, 95)
# x_range = np.linspace(min(data), max(data), 1000)  # Set the range for x-axis
x_range = np.linspace(np.percentile(data, 5), np.percentile(data, 95), 1000)  # Set the range for x-axis
kde_values = kde(x_range)

# Create the plot
fig = go.Figure()

# Add the KDE line
fig.add_trace(go.Scatter(x=x_range, y=kde_values, mode='lines', name='KDE'))

# Customize the layout
fig.update_layout(title='1D KDE Plot',
                  xaxis_title=col_name,
                  yaxis_title='Density')

# Show the plot
fig.show()
